In [1]:
import numpy as np
import sys
sys.path.insert(0, './collecting_standard_features/cleaned/')
import pyximport; pyximport.install()
import nice_utilities
import naive, radial_basis, spherical_coefficients, spherical_harmonics, convert_rascal_coefficients
import unrolling_pca
from nice_utilities import do_full_expansion
from unrolling_pca import UnrollingPCA
from convert_rascal_coefficients import convert_rascal_coefficients, normalize_by_ps
import ClebschGordan, test_utilities

In [2]:
import numpy as np
import ase.io as ase_io
from ase import Atoms
import tqdm
from pathos.multiprocessing import ProcessingPool as Pool




In [3]:
def process_structures(structures):
    for structure in structures: 
        structure.cell =[120, 120, 120]
        structure.positions += np.asarray((60,60,60))
        structure.pbc=True
        structure.wrap()

In [4]:
structures = ase_io.read('collecting_standard_features/cleaned/structures.xyz', index = '0:10')
process_structures(structures)

In [5]:
R_CUT = 6.3

N_MAX = 5
L_MAX = 5


In [6]:
import rascal
from rascal.representations import SphericalInvariants as SOAP
from rascal.representations import SphericalExpansion as SPH
from rascal.neighbourlist.structure_manager import (
        mask_center_atoms_by_species, mask_center_atoms_by_id)


    
    
def get_rascal_coefficients(structures, r_cut, n_max, l_max, n_types, normalize = True):
    HYPERS = {
    'interaction_cutoff': r_cut,
    'max_radial': n_max,
    'max_angular': l_max,
    'gaussian_sigma_constant': 0.05,
    'gaussian_sigma_type': 'Constant',
    'cutoff_smooth_width': 0.3,
    'radial_basis': 'GTO',
    #'expansion_by_species_method' : 'environment wise'
    }
   
    
    sph = SPH(**HYPERS)
    feat = sph.transform(structures).get_features(sph)
    res = convert_rascal_coefficients(feat, n_max, n_types, l_max)
    
    if (normalize):
        normalize_by_ps(res)
    return np.array(res)

In [7]:
def get_rascal_coefficients_parallelized(p, structures, r_cut, n_max, l_max, n_types,
                                         normalize = True, task_size = 100):
    
    tasks = []
    for i in range(0, len(structures), task_size):
        tasks.append([structures[i : i + task_size], r_cut, n_max, l_max, n_types, normalize])
        
    def wrapped(task):
        return get_rascal_coefficients(*task)
    
    result = [res for res in tqdm.tqdm(p.imap(wrapped, tasks), total = len(tasks))]
    return np.concatenate(result, axis = 0)
    

In [8]:
p = Pool(45)

In [9]:
coefficients = get_rascal_coefficients_parallelized(p, structures, R_CUT, N_MAX, L_MAX, 2)
print(coefficients.shape)
coefficients = np.reshape(coefficients, [len(structures), -1, coefficients.shape[1],
                                         coefficients.shape[2], coefficients.shape[3]])
print(coefficients.shape)

100%|██████████| 1/1 [00:00<00:00, 50.26it/s]

(50, 10, 6, 11)
(10, 5, 10, 6, 11)


In [10]:
clebsch = ClebschGordan.ClebschGordan(L_MAX)

In [11]:
def get_values(coefficients):
    even_mask = np.array(range(coefficients.shape[2])) % 2 == 0
   
    odd_mask = np.logical_not(even_mask)
   
    even_1 = np.copy(coefficients)
    even_1[:, :, odd_mask, :] = 0.0

    odd_1 = np.copy(c_coefficients)
    odd_1[:, :, even_mask, :] = 0.0

   
    
    even_2_first, odd_2_first = do_full_expansion(clebsch.precomputed, even_1,
                                              coefficients, L_MAX)

    odd_2_second, even_2_second = do_full_expansion(clebsch.precomputed, odd_1, 
                                                coefficients, L_MAX)
    
    even_2 = np.concatenate([even_2_first, even_2_second], axis = 1)
    odd_2 = np.concatenate([odd_2_first, odd_2_second], axis = 1)
    
    #pca_2_even = UnrollingPCA(n_components = 15, algorithm = 'arpack')
    np.random.seed(42)
    matrix_even = np.random.rand(even_2.shape[1], 15)
    even_2_compressed = np.sum(even_2[:, :, :, :, np.newaxis] * \
                               matrix_even[np.newaxis, :, np.newaxis, np.newaxis, :], axis = 1)
    
   
    np.random.seed(43)
    matrix_odd = np.random.rand(odd_2.shape[1], 15)
    odd_2_compressed = np.sum(odd_2[:, :, :, :, np.newaxis] * \
                               matrix_odd[np.newaxis, :, np.newaxis, np.newaxis, :], axis = 1)
    
    
    even_3_first, odd_3_first = do_full_expansion(clebsch.precomputed, even_2_compressed,
                                              coefficients, L_MAX)

    odd_3_second, even_3_second = do_full_expansion(clebsch.precomputed, odd_2_compressed, 
                                                coefficients, L_MAX)
    
    even_3 = np.concatenate([even_3_first, even_3_second], axis = 1)
    odd_3 = np.concatenate([odd_3_first, odd_3_second], axis = 1)
    
    
    return even_1, odd_1, even_2, odd_2, even_3, odd_3

In [12]:
c_coefficients = coefficients[:, 0]
even_1, odd_1, even_2, odd_2,  even_3, odd_3 = get_values(c_coefficients)

In [13]:
print(even_1.shape)
print(odd_1.shape)
print(even_2.shape)
print(odd_2.shape)
print(even_3.shape)
print(odd_3.shape)

(10, 10, 6, 11)
(10, 10, 6, 11)
(10, 3600, 6, 11)
(10, 3600, 6, 11)
(10, 2160, 6, 11)
(10, 2160, 6, 11)


In [14]:
for structure in structures:
    structure.positions = -structure.positions
    
process_structures(structures)
process_structures(structures)
coefficients = get_rascal_coefficients_parallelized(p, structures, R_CUT, N_MAX, L_MAX, 2)
print(coefficients.shape)
coefficients = np.reshape(coefficients, [len(structures), -1, coefficients.shape[1],
                                         coefficients.shape[2], coefficients.shape[3]])
print(coefficients.shape)

c_coefficients = coefficients[:, 0]
even_1_inverted, odd_1_inverted, even_2_inverted, odd_2_inverted,\
even_3_inverted, odd_3_inverted = get_values(c_coefficients)

100%|██████████| 1/1 [00:00<00:00, 41.55it/s]

(50, 10, 6, 11)
(10, 5, 10, 6, 11)


In [15]:
print(np.sum(np.abs(even_1)))
print(np.sum(np.abs(even_1 - even_1_inverted)))

print(np.sum(np.abs(even_2)))
print(np.sum(np.abs(even_2 - even_2_inverted)))

print(np.sum(np.abs(even_3)))
print(np.sum(np.abs(even_3 - even_3_inverted)))

64.78717966639871
1.6346348704620608e-11
848.8162284040923
4.12983578043376e-10
3450.915115977601
1.414861927721419e-09


In [16]:
print(np.sum(np.abs(odd_1)))
print(np.sum(np.abs(odd_1 + odd_1_inverted)))

print(np.sum(np.abs(odd_2)))
print(np.sum(np.abs(odd_2 + odd_2_inverted)))

print(np.sum(np.abs(odd_3)))
print(np.sum(np.abs(odd_3 + odd_3_inverted)))

64.57055069723128
2.2043962128229753e-11
941.4626608255329
4.662877399309857e-10
3578.5815643291357
1.4169684142788747e-09
